<a href="https://colab.research.google.com/github/ardha27/Real-ESRGAN-Image-Video-Upscaler/blob/main/Real_ESRGAN_Image_and_Video_Upscaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop

from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 720):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
import os

# Define the path to the file
file_path = "/usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py"

# Read the content of the file
with open(file_path, 'r') as file:
    file_data = file.readlines()

# Modify the specific line
for i, line in enumerate(file_data):
    if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
        file_data[i] = 'from torchvision.transforms.functional import rgb_to_grayscale\n'
        break

# Write the modified content back to the file
with open(file_path, 'w') as file:
    file.writelines(file_data)

print("Modification applied successfully.")


In [ ]:
!pip uninstall -y basicsr torch torchvision
!pip install torch==2.1.0 torchvision==0.16.0
!pip install basicsr


In [ ]:
# !pip uninstall -y numpy
# !pip install numpy<2

In [ ]:
# Install versi yang kompatibel (misalnya: 1.24.4)
!pip install numpy==1.24.4

In [ ]:
import os

source = "image" # @param ["image", "video"]
input = "/content/drive/MyDrive/coba1.jpg" # @param {type:"string"}
model = "RealESRGAN_x4plus_anime_6B" # @param ["RealESRGAN_x4plus", "RealESRGAN_x2plus", "RealESRNet_x4plus", "official ESRGAN_x4", "realesr-general-x4v3", "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]
scale = 4 # @param {type:"slider", min:1, max:4, step:1}
face_enhance = True # @param {type:"boolean"}


if source=="image":
  code = "inference_realesrgan.py"
elif source =="video":
  code = "inference_realesrgan_video.py"

def generate_output_path(input_path):
    # Get the directory from the input path
    directory, _ = os.path.split(input_path)

    return directory

def generate_result_path(input_path):
    # Extract the directory and filename from the input path
    directory, filename = os.path.split(input_path)

    # Remove the extension from the filename
    filename_without_extension, extension = os.path.splitext(filename)

    # Create the output path by adding '_out' before the extension
    output_filename = filename_without_extension + "_out" + extension
    output_path = os.path.join(directory, output_filename)

    return output_path

output = generate_output_path(input)
result = generate_result_path(input)
print(output)
print(result)

In [ ]:
if face_enhance == True:
  command = f"python '{code}' -i '{input}' -n '{model}' -o '{output}' -s '{scale}' --face_enhance"
elif face_enhance == False:
  command = f"python '{code}' -i '{input}' -n '{model}' -o '{output}' -s '{scale}'"
!{command}
# -h                   show this help
# -i input-path        input image path (jpg/png/webp) or directory
# -o output-path       output image path (jpg/png/webp) or directory
# -s scale             upscale ratio (can be 2, 3, 4. default=4)
# -t tile-size         tile size (>=32/0=auto, default=0) can be 0,0,0 for multi-gpu
# -m model-path        folder path to the pre-trained models. default=models
# -n model-name        model name (default=realesr-animevideov3, can be realesr-animevideov3 | realesrgan-x4plus | realesrgan-x4plus-anime | realesrnet-x4plus)
# -g gpu-id            gpu device to use (default=auto) can be 0,1,2 for multi-gpu
# -j load:proc:save    thread count for load/proc/save (default=1:2:2) can be 1:2,2,2:2 for multi-gpu
# -x                   enable tta mode"
# -f format            output image format (jpg/png/webp, default=ext/png)
# -v                   verbose output

In [ ]:
# input video
show_video(input)

In [ ]:
# enhanced video
show_video(result)

In [ ]:
from IPython.display import Image, display

display(Image(filename=input, width=720))
display(Image(filename=result, width=720))

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import os
import subprocess
from PIL import Image

# Fungsi bantu untuk hasil path
def generate_result_path(input_path):
    directory, filename = os.path.split(input_path)
    filename_without_extension, extension = os.path.splitext(filename)
    output_filename = filename_without_extension + "_out" + extension
    output_path = os.path.join(directory, output_filename)
    return output_path

# Fungsi resize dengan menjaga rasio
def resize_image(image_path, target_width):
    image = Image.open(image_path)
    w_percent = (target_width / float(image.size[0]))
    h_size = int((float(image.size[1]) * float(w_percent)))
    resized_image = image.resize((target_width, h_size), Image.LANCZOS)
    return resized_image

# Fungsi utama
def enhance_image(input_image, model, scale, face_enhance, display_width):
    input_path = "/content/input_image.png"
    input_image.save(input_path)

    output_path = generate_result_path(input_path)

    command = [
        "python", "inference_realesrgan.py",
        "-i", input_path,
        "-n", model,
        "-s", str(scale),
        "-o", os.path.dirname(output_path)
    ]
    if face_enhance:
        command.append("--face_enhance")

    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        return f"❌ Error during enhancement: {e}", None

    # Resize hasil enhance untuk display
    resized_output = resize_image(output_path, display_width)

    return input_path, resized_output

# Daftar model yang didukung
models = [
    "RealESRGAN_x4plus",
    "RealESRGAN_x2plus",
    "RealESRNet_x4plus",
    "official ESRGAN_x4",
    "realesr-general-x4v3",
    "RealESRGAN_x4plus_anime_6B",
    "realesr-animevideov3"
]

# Resolusi tampilan yang tersedia
display_sizes = [256, 512, 720, 1024, 1280]

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🖼️ Real-ESRGAN Image Enhancer + Display Size Control")
    gr.Markdown("Tingkatkan kualitas gambar dan atur ukuran tampilannya.")

    with gr.Row():
        with gr.Column():
            input_image = gr.Image(label="Input Gambar", type="pil")
            model_choice = gr.Dropdown(choices=models, value="RealESRGAN_x4plus", label="Pilih Model")
            scale_slider = gr.Slider(minimum=1, maximum=4, value=4, step=1, label="Skala (Scale)")
            face_checkbox = gr.Checkbox(label="Face Enhance", value=True)
            display_width = gr.Dropdown(choices=display_sizes, value=720, label="Ukuran Tampilan Output (px)")
            btn = gr.Button("Tingkatkan Gambar")

        with gr.Column():
            input_display = gr.Image(label="Gambar Asli")
            output_display = gr.Image(label="Hasil Enhanced (Resize Only)")

    btn.click(
        fn=enhance_image,
        inputs=[input_image, model_choice, scale_slider, face_checkbox, display_width],
        outputs=[input_display, output_display]
    )

# Jalankan Gradio
demo.launch(debug=True, allowed_paths=["/content"])
